### Classes are Objects

In [1]:
class School:
    def __init__(self, enrollment):
        self.enrollment = enrollment
    
    def enroll_student(self):
        self.enrollment += 1
        
    def graduate_student(self):
        self.enrollment -= 1

In [2]:
s1 = School(enrollment=100)
s2 = School(enrollment=350)

In [3]:
s1, s2 # instances

(<__main__.School at 0x2197abe7050>, <__main__.School at 0x2197abe69d0>)

In [4]:
id(School) # everything in py is an object

2308438394672

In [5]:
SchoolClass = School

In [7]:
SchoolClass(1200)

In [8]:
School.montessori = False

In [9]:

type(s1)

__main__.School

In [10]:
type(School)

type

In [11]:
type(type)

type

### type() as Class Factory

In [12]:
type(1), type(1.2)

(int, float)

In [15]:
type(s1), s1.__class__ # same

(__main__.School, __main__.School)

In [17]:
# type(newClassname, bases, namespace) -> new class

In [18]:
class Student:
    pass

In [20]:
type("School", (), {}) # same as student

__main__.School

In [21]:
class Student:
    major = "undeclared" 

In [22]:
Student2 = type("School", (), {"major": "undeclared"})

In [23]:
s1 = Student()
s2 = Student2()

In [24]:
s1.major, s2.major

('undeclared', 'undeclared')

In [25]:
class Student:
    major = "undeclared"
    
    def greet(self):
        return "Hi, I am student"

In [26]:
Student2 = type("School", (), {"major": "undeclared", "greet": lambda self: "Hi I am student"})

In [27]:
s1 = Student()
s2 = Student2()

In [28]:
s1.greet(), s2.greet()

('Hi, I am student', 'Hi I am student')

In [29]:
class Person:
    pass

class Student(Person):
    major = "undeclared"
    
    def greet(self):
        return "Hi, I am student"

In [30]:
Student2 = type("School", (Person, ), {"major": "undeclared", "greet": lambda self: "Hi I am student"})

In [31]:
Student2.__bases__

(__main__.Person,)

In [32]:
Student.__bases__

(__main__.Person,)

### Defining our own metaclass

In [33]:
# type -> class creating class

In [34]:
class myMetaClass(type):
    pass

In [35]:
class School:
    pass

In [36]:
type(School)

type

In [40]:
class School(metaclass=myMetaClass):
    pass

In [39]:
type(School)

__main__.myMetaClass

In [68]:
class myMetaClass(type):
    
    @classmethod
    def __prepare__(metacls, name, bases):
        print("Metaclass", metacls, "Name",  name, "Bases", bases)
        return {"is_montessori": False, "say_motto": lambda: "say motto"}

In [69]:
class School(metaclass = myMetaClass):
    pass

Metaclass <class '__main__.myMetaClass'> Name School Bases ()


In [70]:
School.is_montessori

False

In [71]:
School.say_motto()

'say motto'

In [72]:
School.__dict__

mappingproxy({'is_montessori': False,
              'say_motto': <function __main__.myMetaClass.__prepare__.<locals>.<lambda>()>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'School' objects>,
              '__weakref__': <attribute '__weakref__' of 'School' objects>,
              '__doc__': None})

### Class creation internals

In [81]:
from typing import Any


class myMetaClass(type):
    
    @classmethod
    def __prepare__(metacls, name, bases): # to create namespace
        print("Metaclass", metacls, "Name",  name, "Bases", bases)
        return {"is_montessori": False, "say_motto": lambda: "say motto"}
    
    @staticmethod
    def __new__(metacls, name, bases, namespace): # create new class and return instance of new class
        print("Class is created here", name, bases, namespace, sep="\n")
        return super().__new__(metacls, name, bases, namespace)
    
    def __init__(cls, name, bases, namespace): # initializing classes
        print("The class is initialized:", cls, name, bases, namespace, sep="\n")
        
    def __call__(self, *args, **kwds): # instantiation
        print(args, kwds)

In [82]:
class School(metaclass=myMetaClass):
    pass

Metaclass <class '__main__.myMetaClass'> Name School Bases ()
Class is created here
School
()
{'is_montessori': False, 'say_motto': <function myMetaClass.__prepare__.<locals>.<lambda> at 0x000002197B35AF20>, '__module__': '__main__', '__qualname__': 'School'}
The class is initialized:
<class '__main__.School'>
School
()
{'is_montessori': False, 'say_motto': <function myMetaClass.__prepare__.<locals>.<lambda> at 0x000002197B35AF20>, '__module__': '__main__', '__qualname__': 'School'}


In [83]:
s1 = School()

() {}


### Zero and 1 Instance classes

In [84]:
class MicroserviceDashboard:
    pass

In [85]:
m1 = MicroserviceDashboard()
m2 = MicroserviceDashboard()

In [86]:
m1, m2

(<__main__.MicroserviceDashboard at 0x2197b2866d0>,
 <__main__.MicroserviceDashboard at 0x2197b271c50>)

In [92]:
class my_meta_class(type): # 0 instance 
    def __call__(self, *args, **kwds):
        raise TypeError("Direct Class instantiation not allowed")

In [93]:
class MicroserviceDashboard(metaclass=my_meta_class):
    pass

In [95]:
MicroserviceDashboard()

TypeError: Direct Class instantiation not allowed

In [96]:
class MicroserviceDashboard(metaclass=my_meta_class):
    @staticmethod
    def running():
        pass
    
    @staticmethod
    def stopped():
        pass
    
    @staticmethod
    def health():
        pass

In [97]:
class my_singleton_meta(type): # singleton
    def __init__(cls, name, bases, namespace):
        cls._instance = None
        super().__init__(name, bases, namespace)
        
    def __call__(cls, *args, **kwds):
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kwds)
            
        return cls._instance

In [99]:
class MicroserviceDashboard(metaclass=my_singleton_meta):
    pass

In [101]:
MicroserviceDashboard()

In [102]:
MicroserviceDashboard()

### Enforcing unique method names

In [123]:
class UniqueConstraintDict(dict):
    def __setitem__(self, key: Any, value: Any) -> None:
        print(f"attempting to set {key} to {value}")
        if key in self:
            raise AttributeError("Attribute name already used! Cannot define class")
        
        super().__setitem__(key, value)
    
class unique_methods_meta(type):
    @classmethod
    def __prepare__(metacls, name, bases):
        print(metacls, name, bases)
        return UniqueConstraintDict()
    
    def __new__(metacls, name, bases, namespace):
        print(namespace)
        return super().__new__(metacls, name, bases, namespace)

In [125]:
class Tractor(metaclass=unique_methods_meta):
    def tow(self, load):
        return f"towing some {load}"
    # def tow(self):
    #     return "towing" 

<class '__main__.unique_methods_meta'> Tractor ()
attempting to set __module__ to __main__
attempting to set __qualname__ to Tractor
attempting to set tow to <function Tractor.tow at 0x000002197B6E1580>
{'__module__': '__main__', '__qualname__': 'Tractor', 'tow': <function Tractor.tow at 0x000002197B6E1580>}
